# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)


import random
import numpy as np
import heapq


def heuristic(path, PoD, PoM):
    return (1 + PoD) * (1 + PoM)

# [todo] Initialize k random state - pick different k sets of states, nCk times. best of all these will be the answer
def set_initial_state(terrain):
    # Selecting random starting point for drone
    initial_state = random.choice(list(terrain.keys()))
    return initial_state

In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

def set_transition_and_cost_matrices(terrain, PoM):
    # Converting node labels to integers for indexing
    node_to_int = {node: i for i, node in enumerate(terrain.keys())}
    num_nodes = len(terrain)

    # Creating a transition matrix based on the adjacency list terrain representation
    transition_matrix = np.zeros((num_nodes, num_nodes))

    for node, neighbors in terrain.items():
        node_index = node_to_int[node]
        total_neighbors = len(neighbors)
        for neighbor in neighbors:
            neighbor_index = node_to_int[neighbor]
            transition_matrix[node_index][neighbor_index] = 1.0 / total_neighbors

    # Creating the cost matrix based on probability of radar presence (PoM)
    cost_matrix = np.full((num_nodes, num_nodes), PoM)

    return transition_matrix, cost_matrix

In [3]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented


def successor_function(current_state, terrain, transition_matrix):
    # Get the neighbors (successor states) of the current state based on the transition matrix
    successors = []
    for neighbor in terrain[current_state]:
        # Use the transition probabilities from the transition matrix to determine the probability of moving to the neighbor
        probability = transition_matrix[current_state][neighbor]
        successors.append((neighbor, probability))

    return successors

def generate_successors(path, terrain):
    # Simulate the drone's movement by generating possible successors for the current path
    successors = []
    for neighbor in terrain[path[-1]]:
        successors.append(path + [neighbor])
    return successors

In [4]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def is_goal_state(current_state, destination):
    # Checking if current state is same as destination
    return current_state == destination

### 2.	Definition of Algorithm 1 (Local Beam Search)

In [5]:
#Code Block : Function for algorithm 1 implementation

def local_beam_search(k, terrain, PoD, PoM, num_iterations):
    # Creating k beams of random paths.
    beams = [[random.choice(list(terrain.keys()))] for _ in range(k)]

    for _ in range(num_iterations):
        successors = []
        for path in beams:
            successors.extend(generate_successors(path, terrain))

        # Analyze each successor's heuristic value.
        successors_with_h = [(successor, heuristic(successor, PoD, PoM)) for successor in successors]
        # The top k successors are chosen to act as new beams after the successors are sorted by heuristic value.
        beams = [path for path, h in sorted(successors_with_h, key=lambda x: x[1])[:k]]

    return beams

### 3.	Definition of Algorithm 2 (A* Search)

In [6]:
#Code Block : Function for algorithm 2 implementation

def a_star_search(terrain, PoD, PoM, start, goal):
    #Priority queue (heap) A* search
    frontier = [(0, start)]
    explored = set()
    g = {start: 0}
    parents = {start: None}

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            # Reconstruct the path from start to goal
            path = [current]
            while parents[current] is not None:
                current = parents[current]
                path.append(current)
            return list(reversed(path))

        explored.add(current)

        for neighbor in terrain[current]:
            cost = heuristic([neighbor], PoD, PoM)
            new_g = g[current] + cost

            if neighbor not in g or new_g < g[neighbor]:
                g[neighbor] = new_g
                f = new_g + heuristic([neighbor], PoD, PoM)
                heapq.heappush(frontier, (f, neighbor))
                parents[neighbor] = current

    return None

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [7]:
#Code Block : Function & call to get inputs (start/end state)

#[todo] Figure out if input is start and end, or values inside grid
def get_user_inputs():
    # to get user input on start state and end state
    start = input("Enter the start state (current location): ")
    goal = input("Enter the goal state (destination): ")
    return start, goal

# Sample terrain as an adjacency list representation of the maze
terrain = {
    'A': ['B', 'C'],
    'B': ['A', 'D'],
    'C': ['A', 'D'],
    'D': ['B', 'C', 'E'],
    'E': ['D', 'F'],
    'F': ['E', 'G'],
    'G': ['F']
}

# (land terrain – the probability of detection (PoD), drone (probability of radar (PoM)
PoD = 0.8
PoM = 0.2
k = 3
num_iterations = 10

n = len(terrain)
N = n  # worst case scenario (time-complexity)


# user inputs for start state and end state
start, goal = get_user_inputs()

# initialize initial state
initial_state = start

# Setting up transition and cost matrices
transition_matrix, cost_matrix = set_transition_and_cost_matrices(terrain, PoM)


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [8]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Local Beam Search to find best paths
best_paths = local_beam_search(k, terrain, PoD, PoM, num_iterations)

print("\n---- Local Beam Search Results -----")
for i, path in enumerate(best_paths):
    total_cost = heuristic(path, PoD, PoM)
    print(f"Beam {i + 1}: Path {path} (Cost: {total_cost})")



---- Local Beam Search Results -----
Beam 1: Path ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A'] (Cost: 2.16)
Beam 2: Path ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'D'] (Cost: 2.16)
Beam 3: Path ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'C', 'A'] (Cost: 2.16)


In [18]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))


# A* search to find optimal path
optimal_path = a_star_search(terrain, PoD, PoM, start, goal)

print("\n------- A* Search Results -------")
if optimal_path:
    total_cost = heuristic(optimal_path, PoD, PoM)
    print(f"Optimal Path: {optimal_path} (Total Cost: {total_cost})")
else:
    print("Goal could not be reached. No path found.")


------- A* Search Results -------
Optimal Path: ['A', 'B', 'D', 'E', 'F', 'G'] (Total Cost: 2.16)


### 5.	Comparitive Analysis

In [23]:
#Code Block : Print the Time & Space complexity of algorithm 1

def calculate_time_complexity_local_beam_search(num_iterations, k, n):
    # Time complexity for Local Beam Search
    time_complexity = num_iterations * k * n * np.log(k * n)

    # Space complexity for Local Beam Search
    space_complexity = k * n

    return time_complexity, space_complexity

local_beam_search_time, space_complexity = calculate_time_complexity_local_beam_search(num_iterations, k, n)
print(f"Time Complexity of Local Beam Search: O({local_beam_search_time})")
print(f"Local Beam Search Space Complexity: O({local_beam_search_space})")


Time Complexity of Local Beam Search: O(639.3497119219188)
Local Beam Search Space Complexity: O(158.1953125)


In [24]:
#Code Block : Print the Time & Space complexity of algorithm 2

def calculate_time_complexity_a_star_search(n, N):
    # Time complexity for A* Search
    time_complexity = n * np.log(N)

    # Space complexity for A* Search
    space_complexity = N

    return time_complexity, space_complexity

a_star_search_time, space_complexity = calculate_time_complexity_a_star_search(n, N)
print(f"Time Complexity of A* Search: O({a_star_search_time})")
print(f"A* Search Space Complexity: O({space_complexity})")

Time Complexity of A* Search: O(13.621371043387192)
A* Search Space Complexity: O(7)


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________